In [1]:
import geopandas as gpd
from shapely.geometry import Polygon
import h3

# Load Texas boundary
states = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_500k.zip")
texas = states[states['NAME'] == 'Texas'].to_crs(epsg=4326)

# Merge multipolygons into one
texas_poly = texas.geometry.unary_union
if texas_poly.geom_type == "MultiPolygon":
    texas_poly = max(texas_poly.geoms, key=lambda a: a.area)

# Extract coordinates (lon, lat)
boundary = list(texas_poly.exterior.coords)

# Convert boundary to LatLngPoly (lat, lon order)
latlng_boundary = [(lat, lon) for lon, lat in boundary]  # note the swap
polygon = h3.LatLngPoly(latlng_boundary)

# H3 resolution
resolution = 6

# Generate H3 hexes
hexes = h3.polygon_to_cells(polygon, resolution)

# Convert H3 hexes to Shapely polygons
hex_polys = [
    Polygon([(lon, lat) for lat, lon in h3.cell_to_boundary(h)])
    for h in hexes
]

# Make GeoDataFrame
hex_df = gpd.GeoDataFrame({'h3_index': list(hexes)}, geometry=hex_polys, crs='EPSG:4326')

hex_df = hex_df.to_crs(epsg=32614)

roadDf = gpd.read_parquet("../../data/raw/texas_edges.parquet", columns=["geometry"])

roadDf = roadDf.to_crs(epsg=32614)

sindex = hex_df.sindex
possible_idx = list(sindex.intersection(roadDf.total_bounds))
hex_df_subset = hex_df.iloc[possible_idx]
hexesWithCells = gpd.sjoin(hex_df_subset, roadDf, predicate="intersects")

hexesWithCells = hexesWithCells.drop(columns=["index_right"])

hexesWithCells.to_parquet("../../data/processed/cells.parquet")

C:\Users\mahd\AppData\Local\Temp\ipykernel_3116\2257001302.py:10: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  texas_poly = texas.geometry.unary_union


In [2]:
# hex_df is all hexes, hexesWithCells is only hexes with roads
hexes_no_roads = hex_df[~hex_df['h3_index'].isin(hexesWithCells['h3_index'])]

print("Number of hexes without roads:", len(hexes_no_roads))

Number of hexes without roads: 802


In [ ]:
import matplotlib.pyplot as plt

# Plot the whole state boundary
fig, ax = plt.subplots(figsize=(10, 10))
texas.plot(ax=ax, color='none', edgecolor='black')

# Plot all hexes (you can pick which — all, with roads, or without)
hexesWithCells.to_crs(epsg=4326).plot(ax=ax, alpha=0.5, edgecolor='gray')

ax.set_title("All Texas Hexes with Roads")
plt.show()
